In [1]:
# !pip install livelossplot -q
# !pip install bposd -q
# !pip install json -q

In [2]:
import os
import sys
sys.path.append(os.getcwd() + "/src")

In [3]:
from global_params import params

error_rates = params['eval_p_range']
n_samples = 50

## Benchmark random CPC codes

In [4]:
from scoring.score_dataset import run_decoder
import json
from CPC import generate_random
import numpy as np
import utils

best_word_succ_rates = [0.0] * len(error_rates)
best_pc = [[]] * len(error_rates)

for i, e in enumerate(error_rates):
	for _ in range(n_samples):
		pc, _, _, _ = generate_random.random_cpc()
		wsr = run_decoder(pc, np.ones(pc.shape[-1]) * e)
		if wsr > best_word_succ_rates[i]:
			print(f"error rate {e} and wsr {wsr} is better than {best_word_succ_rates[i]}")
			best_word_succ_rates[i] = wsr
			best_pc[i] = pc

out_json = {'random_cpc': {}}
for i, r in enumerate(best_word_succ_rates):
	out_json['random_cpc'][f"error_rate_{error_rates[i]}"] = r
	out_json['random_cpc'][f"error_rate_{error_rates[i]}_pc"] = best_pc[i]


with open(utils.get_eval_baseline_path(), "w") as outfile:
    print("WRITING", out_json)
    json.dump(out_json, outfile, cls=utils.NpEncoder)

error rate 0.005 and wsr 0.97832 is better than 0.0
error rate 0.005 and wsr 0.98664 is better than 0.97832
error rate 0.005 and wsr 0.98894 is better than 0.98664
error rate 0.005 and wsr 0.9896 is better than 0.98894
WRITING {'random_cpc': {'error_rate_0.005': 0.9896, 'error_rate_0.005_pc': array([[0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0,
        1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
        1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
        1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
        1, 1, 0, 

## Benchmark AI learned codes

In [5]:
from training_qec import evaluate_performance
from generating import generating_model as gen_model
from scoring import score_model
from global_params import params
import utils
import torch
# TODO: use OSD for eval
N_dec = 8  # Changed from 6
h = 12  # changed from 8
d_model = 60

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

scoring_model = score_model.ScoringTransformer(
    params['n_data_qubits'], params['n_check_qubits'], h, d_model, N_dec, device, dropout=0).to(device)

p = os.path.join(
    utils.get_best_scoring_model_path())
scoring_model.load_state_dict(torch.load(p, map_location=device))

generating_model = gen_model.GeneratingModel(
    device=device,
)

p_eval_range = params['eval_p_range']
evaluate_performance(scoring_model, generating_model, p_eval_range, epoch=100, is_init_epoch=False, averaging_samples=n_samples, n_tests=1_000_000)

cpu
Starting to evaluate performance for epoch 100 with file eval/results_bsc_iid_noise_(15,0).json
Best performance for p=0.005 is 0.9856
Best performance for p=0.005 is 0.9856
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p=0.005 is 0.98764
Best performance for p